<a href="https://colab.research.google.com/github/mdhornstein/computer-vision-interpretability-paleontology-v0/blob/main/notebooks/01_extract_model_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# --- 1. Setup ---
import os, sys, glob, warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from keras.models import load_model, Model
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input, decode_predictions
from keras.preprocessing import image

In [3]:
# --- 2. Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# Path to your files
base_dir = "/content/drive/MyDrive/cv_paleontology"
model_archive = os.path.join(base_dir, "inception_resnet_net_v2_model.7z")
dataset_dir = os.path.join(base_dir, "reduced-FID")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# --- 3. Extract model archive ---
!sudo apt-get install p7zip-full -y
!7z x "{model_archive}" -o"{base_dir}/model"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/cv_paleontology/                                                 1 file, 582014617 bytes (556 MiB)

Extracting archive: /content/drive/MyDrive/cv_paleontology/inception_resnet_net_v2_model.7z
--
Path = /content/drive/MyDrive/cv_paleontology/inception_resnet_net_v2_model.7z
Type = 7z
Physical Size = 582014617
Headers Size = 280
Method = LZMA2:25
Solid = +
Blocks = 1

  0%      0% 1        0% 1 - in

In [4]:
!ls -l "{base_dir}/model"

total 4
drwx------ 2 root root 4096 Aug 27 07:10 'inception_resnet_net_v2 model'


In [7]:
!mv "{base_dir}/model/inception_resnet_net_v2 model" "{base_dir}/model/inception_resnet_net_v2_model"

In [8]:
!ls "{base_dir}/model/inception_resnet_net_v2_model"

model.ckpt-60.data-00000-of-00001  model.ckpt-60.index	model.ckpt-60.meta


In [18]:
print("--- {base_dir}: ---")
!ls "{base_dir}"
print()
print("--- {base_dir}/model: ---")
!ls "{base_dir}/model"

--- {base_dir}: ---
data  inception_resnet_net_v2_model.7z	model

--- {base_dir}/model: ---
inception_resnet_net_v2_model


In [ ]:
# --- 4. Load the model ---
model_path = os.path.join(base_dir, "model", "inception_resnet_net_v2_model")
latest_checkpoint = tf.train.latest_checkpoint(model_path)

# Load the model
# You might need to rebuild the model architecture first if you only have the checkpoint files.
# For this example, let's assume you have a saved model file or you can rebuild it.
# If you have a .h5 file, you can use:
# model = load_model(os.path.join(model_path, "your_model_file.h5"))

# If you only have checkpoint files, you'll need to restore from checkpoint
# This requires knowing the original model architecture.
# Example (assuming you have the original model definition):
# model = your_model_architecture() # Define or load your model architecture
# model.load_weights(latest_checkpoint)

# For demonstration purposes, let's load a pre-trained InceptionResNetV2 model
# and then attempt to load weights from the checkpoint.
# Note: This might not work directly if the checkpoint is from a fine-tuned model
# with a different head than the standard InceptionResNetV2.

base_model = InceptionResNetV2(weights='imagenet', include_top=False)
x = base_model.output
# Add your custom layers here if the checkpoint is from a fine-tuned model
# For example:
# x = tf.keras.layers.GlobalAveragePooling2D()(x)
# predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x) # num_classes is the number of classes in your task
# model = Model(inputs=base_model.input, outputs=predictions)

# Attempt to load weights from the checkpoint
try:
    model = Model(inputs=base_model.input, outputs=base_model.output) # Create a temporary model to load weights
    model.load_weights(latest_checkpoint)
    print("Successfully loaded weights from checkpoint.")
except Exception as e:
    print(f"Could not load weights from checkpoint: {e}")
    print("Loading a standard InceptionResNetV2 model with ImageNet weights instead.")
    model = InceptionResNetV2(weights='imagenet', include_top=True)


model.summary()

**Note:** The code above attempts to load weights from the checkpoint. If the checkpoint is from a fine-tuned model with a different output layer than the standard InceptionResNetV2, you will need to define the correct model architecture before loading the weights.